# 🤖 LangGraph 多 Agent 系统实战

本 Notebook 展示如何使用 LangGraph 构建复杂的多 Agent 协作系统。

## 📋 学习目标

- LangGraph 基础概念（State、Node、Edge）
- 构建状态机工作流
- 多 Agent 协作模式
- 条件路由和决策
- 实战案例：智能内容创作团队


## 📦 安装依赖


In [1]:
%pip install -q langgraph langchain langchain-openai
%pip install -q sentence-transformers


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 🔧 配置 DeepSeek API


In [2]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_openai import ChatOpenAI

print("🔧 配置 DeepSeek API")
print("="*60)
print("本 Notebook 使用 DeepSeek 模型（兼容 OpenAI API）")
print("\n📝 获取 API Key:")
print("1. 访问 https://platform.deepseek.com/")
print("2. 注册/登录账号")
print("3. 创建 API Key")
print("\n💰 费用说明:")
print("DeepSeek 比 OpenAI 便宜很多，适合学习和开发")
print("="*60)

# 方式1：在终端中设置环境变量（推荐）
# export OPENAI_API_KEY="your-deepseek-api-key-here"
# export OPENAI_API_BASE="https://api.deepseek.com"

# 方式2：临时在 Notebook 中设置（仅本次运行有效，不会影响系统环境变量）
# 如果你没有在终端设置，可以取消下面两行的注释并填入你的 API Key
# os.environ["OPENAI_API_KEY"] = "your-deepseek-api-key-here"
# os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

# 检查配置
api_key = os.environ.get("OPENAI_API_KEY", "")
api_base = os.environ.get("OPENAI_API_BASE", "https://api.deepseek.com")

if not api_key or api_key == "your-deepseek-api-key-here":
    print("\n⚠️  警告: 未检测到有效的 API Key！")
    print("\n请选择以下方式之一配置：")
    print("1. 在终端运行：")
    print("   export OPENAI_API_KEY='your-actual-api-key'")
    print("   export OPENAI_API_BASE='https://api.deepseek.com'")
    print("\n2. 或在上方取消注释并填入你的 API Key")
else:
    # 确保 API Base 设置正确
    if not api_base or api_base == "https://api.openai.com":
        os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"
        print(f"\n✅ API Key 已配置")
        print(f"✅ API Base 已设置为: https://api.deepseek.com")
    else:
        print(f"\n✅ API Key 已配置")
        print(f"✅ API Base: {api_base}")

llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_base="https://api.deepseek.com",
    temperature=0.7
)

print("✅ 使用模型: deepseek-chat")


🔧 配置 DeepSeek API
本 Notebook 使用 DeepSeek 模型（兼容 OpenAI API）

📝 获取 API Key:
1. 访问 https://platform.deepseek.com/
2. 注册/登录账号
3. 创建 API Key

💰 费用说明:
DeepSeek 比 OpenAI 便宜很多，适合学习和开发

✅ API Key 已配置
✅ API Base: https://api.deepseek.com
✅ 使用模型: deepseek-chat


## 一、LangGraph 基础概念

### 核心概念

```
State (状态)
   ↓
Node (节点/Agent)
   ↓
Edge (边/转换)
   ↓
Graph (整个工作流)
```

### 简单示例


In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# 1. 定义状态
class SimpleState(TypedDict):
    messages: list[str]
    counter: int

# 2. 定义节点函数
def step1(state: SimpleState):
    print("执行步骤 1")
    return {
        "messages": state["messages"] + ["步骤1完成"],
        "counter": state["counter"] + 1
    }

def step2(state: SimpleState):
    print("执行步骤 2")
    return {
        "messages": state["messages"] + ["步骤2完成"],
        "counter": state["counter"] + 1
    }

# 3. 构建图
workflow = StateGraph(SimpleState)
workflow.add_node("step1", step1)
workflow.add_node("step2", step2)

# 4. 添加边
workflow.set_entry_point("step1")
workflow.add_edge("step1", "step2")
workflow.add_edge("step2", END)

# 5. 编译并运行
app = workflow.compile()

result = app.invoke({"messages": [], "counter": 0})
print("\n结果:")
print(f"消息: {result['messages']}")
print(f"计数: {result['counter']}")


执行步骤 1
执行步骤 2

结果:
消息: ['步骤1完成', '步骤2完成']
计数: 2


## 二、监控和调试 Agent 执行过程

LangGraph 提供多种方式监控 Agent 的执行过程，帮助你了解每一步在做什么。


### 方法1: 在 Agent 内部打印日志（最简单）


In [ ]:
# 在 Agent 函数中添加详细的打印语句
def monitored_agent(state):
    print("\n" + "="*60)
    print("🔍 [Agent 开始执行]")
    print(f"📥 输入状态: {state}")
    print("="*60)
    
    # 执行逻辑
    result = {
        "messages": state["messages"] + ["处理完成"],
        "counter": state["counter"] + 1
    }
    
    print("\n" + "="*60)
    print("✅ [Agent 执行完成]")
    print(f"📤 输出状态: {result}")
    print("="*60 + "\n")
    
    return result

# 测试
test_state = {"messages": ["开始"], "counter": 0}
result = monitored_agent(test_state)


### 方法2: 使用 LangGraph 的 Stream 模式（推荐）


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class DemoState(TypedDict):
    messages: list[str]
    counter: int

def agent_a(state: DemoState):
    print("  🅰️ Agent A 正在执行...")
    return {
        "messages": state["messages"] + ["A处理"],
        "counter": state["counter"] + 1
    }

def agent_b(state: DemoState):
    print("  🅱️ Agent B 正在执行...")
    return {
        "messages": state["messages"] + ["B处理"],
        "counter": state["counter"] + 1
    }

# 构建工作流
workflow = StateGraph(DemoState)
workflow.add_node("agent_a", agent_a)
workflow.add_node("agent_b", agent_b)
workflow.set_entry_point("agent_a")
workflow.add_edge("agent_a", "agent_b")
workflow.add_edge("agent_b", END)
app = workflow.compile()

# 使用 stream 模式查看每一步的执行
print("\n🔍 Stream 模式 - 实时查看每个节点的执行:")
print("="*60 + "\n")

for step in app.stream({"messages": [], "counter": 0}):
    print(f"📍 当前步骤: {list(step.keys())[0]}")
    print(f"📊 状态更新: {step}")
    print("-"*60)


### 方法3: 使用自定义回调处理器（最灵活）


In [ ]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult
from typing import Any, Dict, List

class DetailedCallbackHandler(BaseCallbackHandler):
    """自定义回调处理器，监控每个 LLM 调用"""
    
    def __init__(self):
        self.step_count = 0
        
    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs) -> None:
        self.step_count += 1
        print(f"\n{'='*60}")
        print(f"🚀 [步骤 {self.step_count}] LLM 开始调用")
        print(f"{'='*60}")
        print(f"📝 Prompt:")
        for i, prompt in enumerate(prompts):
            print(f"\n提示 {i+1}:")
            print(prompt[:200] + "..." if len(prompt) > 200 else prompt)
        print("-"*60)
    
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"\n✅ [步骤 {self.step_count}] LLM 调用完成")
        print(f"💬 响应: {response.generations[0][0].text[:200]}...")
        print(f"{'='*60}\n")
    
    def on_llm_error(self, error: Exception, **kwargs) -> None:
        print(f"\n❌ [步骤 {self.step_count}] LLM 调用出错: {error}")
    
    def on_chain_start(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs) -> None:
        print(f"\n🔗 Chain 开始: {serialized.get('name', 'Unknown')}")
        print(f"📥 输入: {inputs}")
    
    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"✅ Chain 完成")
        print(f"📤 输出: {str(outputs)[:200]}...")

# 使用回调处理器
callback = DetailedCallbackHandler()

# 在 LLM 调用时传入回调
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm_with_callback = ChatOpenAI(
    model="deepseek-chat",
    openai_api_base="https://api.deepseek.com",
    temperature=0,
    callbacks=[callback]  # 传入回调处理器
)

prompt = ChatPromptTemplate.from_template("简短介绍一下 {topic}")
chain = prompt | llm_with_callback

print("\n🎬 开始执行链...")
result = chain.invoke({"topic": "人工智能"})


### 方法4: 使用 LangSmith 平台（生产环境推荐）

LangSmith 是 LangChain 官方的监控和调试平台，提供可视化界面。

```python
# 1. 安装 LangSmith
# pip install langsmith

# 2. 设置环境变量
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-langsmith-api-key"
os.environ["LANGCHAIN_PROJECT"] = "multi-agent-demo"

# 3. 运行代码，自动上传到 LangSmith
# 访问 https://smith.langchain.com/ 查看执行详情
```

**LangSmith 功能：**
- 🔍 **可视化工作流**：图形化展示 Agent 执行路径
- ⏱️ **性能分析**：每个节点的耗时统计
- 💰 **成本追踪**：Token 使用和费用统计
- 🐛 **错误追踪**：详细的错误堆栈和上下文
- 📊 **数据集测试**：批量测试和评估

**获取 API Key：**
1. 访问 [https://smith.langchain.com/](https://smith.langchain.com/)
2. 注册账号
3. 在设置中创建 API Key


### 方法5: 完整的监控示例 - 多 Agent 系统

结合以上方法，创建一个完整的监控系统：


In [ ]:
import time
from datetime import datetime

class MonitoredAgentState(TypedDict):
    task: str
    results: list
    execution_log: list

def log_execution(agent_name: str, state: dict, action: str):
    """记录执行日志"""
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    log_entry = f"[{timestamp}] {agent_name} - {action}"
    print(f"  {log_entry}")
    return log_entry

def researcher_monitored(state: MonitoredAgentState):
    """研究员 Agent - 带监控"""
    agent_name = "🔬 研究员"
    
    # 开始执行
    log = log_execution(agent_name, state, "开始研究")
    state["execution_log"].append(log)
    
    # 模拟工作
    time.sleep(0.5)
    print(f"    💭 分析任务: {state['task']}")
    result = f"关于'{state['task']}'的研究报告"
    
    # 完成执行
    log = log_execution(agent_name, state, f"完成研究，生成报告")
    state["execution_log"].append(log)
    
    return {
        **state,
        "results": state["results"] + [result]
    }

def writer_monitored(state: MonitoredAgentState):
    """作家 Agent - 带监控"""
    agent_name = "✍️ 作家"
    
    log = log_execution(agent_name, state, "开始写作")
    state["execution_log"].append(log)
    
    time.sleep(0.5)
    print(f"    💭 基于: {state['results'][-1]}")
    result = f"文章: {state['task']}"
    
    log = log_execution(agent_name, state, "完成写作")
    state["execution_log"].append(log)
    
    return {
        **state,
        "results": state["results"] + [result]
    }

def editor_monitored(state: MonitoredAgentState):
    """编辑 Agent - 带监控"""
    agent_name = "📝 编辑"
    
    log = log_execution(agent_name, state, "开始审核")
    state["execution_log"].append(log)
    
    time.sleep(0.5)
    print(f"    💭 审核: {state['results'][-1]}")
    result = f"最终版本: {state['task']}"
    
    log = log_execution(agent_name, state, "完成审核")
    state["execution_log"].append(log)
    
    return {
        **state,
        "results": state["results"] + [result]
    }

# 构建工作流
workflow_monitored = StateGraph(MonitoredAgentState)
workflow_monitored.add_node("researcher", researcher_monitored)
workflow_monitored.add_node("writer", writer_monitored)
workflow_monitored.add_node("editor", editor_monitored)

workflow_monitored.set_entry_point("researcher")
workflow_monitored.add_edge("researcher", "writer")
workflow_monitored.add_edge("writer", "editor")
workflow_monitored.add_edge("editor", END)

app_monitored = workflow_monitored.compile()

# 执行并监控
print("\n" + "="*60)
print("🚀 启动多 Agent 监控系统")
print("="*60 + "\n")

initial_state = {
    "task": "机器学习",
    "results": [],
    "execution_log": []
}

# 使用 stream 模式实时查看
for i, step in enumerate(app_monitored.stream(initial_state), 1):
    print(f"\n📍 步骤 {i}: {list(step.keys())[0]}")
    print("-"*60)

print("\n" + "="*60)
print("✅ 执行完成")
print("="*60)

# 打印执行日志
print("\n📋 完整执行日志:")
for log in step[list(step.keys())[0]]["execution_log"]:
    print(f"  {log}")

print(f"\n📊 最终结果:")
for i, result in enumerate(step[list(step.keys())[0]]["results"], 1):
    print(f"  {i}. {result}")


### 📊 监控方法对比

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| **Agent 内打印** | 简单直接 | 代码侵入性强 | 快速调试 |
| **Stream 模式** | 实时、无侵入 | 只能看到状态变化 | 开发调试 |
| **自定义回调** | 灵活、详细 | 需要编写回调类 | 复杂监控需求 |
| **LangSmith** | 可视化、功能强大 | 需要注册账号 | 生产环境 |
| **组合方案** | 全面监控 | 输出较多 | 深度调试 |

### 💡 推荐方案

**开发阶段：**
```python
# 组合使用 print + stream
for step in app.stream(state):
    print(f"节点: {list(step.keys())[0]}")
    print(f"状态: {step}")
```

**生产环境：**
```python
# 启用 LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-key"
```

**深度调试：**
```python
# 自定义回调 + 详细日志
callback = DetailedCallbackHandler()
llm = ChatOpenAI(..., callbacks=[callback])
```


### 2.1 定义状态


In [4]:
from typing import TypedDict, List
import operator
from langchain.schema import HumanMessage, AIMessage

class AgentState(TypedDict):
    topic: str
    research_notes: str
    draft_content: str
    final_content: str
    messages: Annotated[List, operator.add]
    next_agent: str
    iteration: int
    max_iterations: int


### 2.2 创建各个 Agent


In [5]:
from langchain.prompts import ChatPromptTemplate

# 研究员 Agent
def researcher_agent(state: AgentState) -> AgentState:
    print("\n📚 研究员 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的研究员。请针对以下主题进行研究，提供关键要点和有价值的信息。

主题：{topic}

请提供：
1. 核心概念定义
2. 重要特点（3-5个）
3. 实际应用场景
4. 注意事项

研究笔记："""
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": state["topic"]})
    
    return {
        **state,
        "research_notes": result.content,
        "messages": [f"[研究员] 完成研究: {state['topic'][:50]}..."],
        "next_agent": "writer"
    }

# 作家 Agent
def writer_agent(state: AgentState) -> AgentState:
    print("\n✍️ 作家 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的技术作家。基于研究员提供的笔记，撰写一篇结构清晰、通俗易懂的文章。

主题：{topic}

研究笔记：
{research_notes}

要求：
- 使用清晰的标题和段落结构
- 语言通俗易懂，适合初学者
- 包含实例说明
- 字数约500字

文章内容："""
    )
    
    chain = prompt | llm
    result = chain.invoke({
        "topic": state["topic"],
        "research_notes": state["research_notes"]
    })
    
    return {
        **state,
        "draft_content": result.content,
        "messages": [f"[作家] 完成初稿撰写"],
        "next_agent": "editor"
    }

# 编辑 Agent
def editor_agent(state: AgentState) -> AgentState:
    print("\n🔍 编辑 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的编辑。请审核并改进以下文章。

原文：
{draft_content}

请：
1. 检查语法和表达
2. 优化结构和逻辑
3. 确保内容准确性
4. 增强可读性

如果文章质量已经很好，保持原样并添加简短评价。
如果需要改进，请直接输出改进后的版本。

最终文章："""
    )
    
    chain = prompt | llm
    result = chain.invoke({"draft_content": state["draft_content"]})
    
    new_iteration = state["iteration"] + 1
    
    # 决定是否需要再次迭代
    if new_iteration >= state["max_iterations"]:
        next_agent = "END"
    else:
        # 可以根据质量评估决定是否继续
        next_agent = "END"  # 简化版本，一次迭代后结束
    
    return {
        **state,
        "final_content": result.content,
        "messages": [f"[编辑] 完成审核和改进"],
        "next_agent": next_agent,
        "iteration": new_iteration
    }

print("✅ Agent 定义完成！")


✅ Agent 定义完成！


In [6]:
from langgraph.graph import StateGraph, END

# 路由函数：决定下一个节点
def route_agent(state: AgentState) -> str:
    next_agent = state.get("next_agent", "researcher")
    if next_agent == "END":
        return END
    return next_agent

# 创建图
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("researcher", researcher_agent)
workflow.add_node("writer", writer_agent)
workflow.add_node("editor", editor_agent)

# 设置入口
workflow.set_entry_point("researcher")

# 添加条件边
workflow.add_conditional_edges(
    "researcher",
    route_agent,
    {"writer": "writer", END: END}
)

workflow.add_conditional_edges(
    "writer",
    route_agent,
    {"editor": "editor", END: END}
)

workflow.add_conditional_edges(
    "editor",
    route_agent,
    {"writer": "writer", END: END}
)

# 编译
app = workflow.compile()

print("✅ 工作流构建完成！")


✅ 工作流构建完成！


### 2.4 运行多 Agent 系统


In [7]:
# 初始化状态
initial_state = {
    "topic": "深度学习中的 Transformer 架构",
    "research_notes": "",
    "draft_content": "",
    "final_content": "",
    "messages": [],
    "next_agent": "researcher",
    "iteration": 0,
    "max_iterations": 2
}

print("🚀 启动多 Agent 内容创作系统...")
print(f"📝 主题: {initial_state['topic']}\n")
print("="*60)

# 运行工作流
final_state = app.invoke(initial_state)

print("\n" + "="*60)
print("\n🎉 创作完成！")
print("\n📋 工作流程:")
for msg in final_state["messages"]:
    print(f"  {msg}")

print("\n" + "="*60)
print("\n📚 研究笔记:")
print(final_state["research_notes"][:300] + "...")

print("\n" + "="*60)
print("\n📄 最终文章:")
print(final_state["final_content"])


🚀 启动多 Agent 内容创作系统...
📝 主题: 深度学习中的 Transformer 架构


📚 研究员 Agent 工作中...

✍️ 作家 Agent 工作中...

🔍 编辑 Agent 工作中...


🎉 创作完成！

📋 工作流程:
  [研究员] 完成研究: 深度学习中的 Transformer 架构...
  [作家] 完成初稿撰写
  [编辑] 完成审核和改进


📚 研究笔记:
好的，收到您的研究需求。作为一名专业研究员，我将针对“深度学习中的 Transformer 架构”这一主题，为您提供一份结构清晰、信息详实的研究笔记。

---

### **研究主题：深度学习中的 Transformer 架构**

**摘要**：Transformer 是一种基于自注意力机制的深度学习模型架构，由谷歌团队在 2017 年的里程碑式论文《Attention Is All You Need》中首次提出。它彻底改变了自然语言处理领域的格局，并逐渐扩展到计算机视觉、音频处理等多个领域，成为当今许多前沿人工智能模型的核心基础。

---

#### **1. 核心概念定义**

**...


📄 最终文章:
### **Transformer：重塑AI格局的“注意力”架构**

在人工智能领域，一个名为 **Transformer** 的模型架构正日益成为众多应用的核心驱动力——从智能对话、实时翻译到图像生成，其影响力无处不在。它被誉为深度学习的一次革命，彻底改变了机器理解与生成信息的方式。

#### **一、什么是Transformer？**

简单来说，Transformer 是一种专门处理序列数据（如一句话、一段代码）的深度学习模型。其核心思想非常直观：**在处理信息时，学会“集中注意力”**。

试想你在阅读这句话：“苹果很好吃，但最新发布的苹果手机很贵。”当看到第二个“苹果”时，你会自然联想到它指的是品牌而非水果。这种能力源于你对上下文的整体把握，以及对不同词语赋予的不同“注意力”。

Transformer 正是借鉴了这种机制。它通过一项称为 **“自注意力”** 的技术，使模型在处理序列中任一元素（如单词）时，能够同时关注序列中的所有其他部分，从而捕捉全局的上下文关联。

#### **二、Transformer为何如此强大？**


## 三、高级特性：动态路由和循环

### 3.1 带质量检查的循环改进


In [8]:
# 质量评估 Agent
def quality_checker(state: AgentState) -> AgentState:
    print("\n🎯 质量检查 Agent 工作中...")
    
    prompt = ChatPromptTemplate.from_template(
        """评估以下文章的质量，并给出1-10的分数。

文章：
{content}

请只回答一个数字（1-10）和简短评价（一句话）。
格式：分数: X, 评价: XXX
"""
    )
    
    chain = prompt | llm
    result = chain.invoke({"content": state["final_content"]})
    
    # 简单解析分数
    try:
        score_str = result.content.split("分数:")[1].split(",")[0].strip()
        score = int(score_str)
    except:
        score = 8  # 默认分数
    
    # 如果分数低于8且未超过最大迭代次数，返回给作家改进
    new_iteration = state["iteration"] + 1
    if score < 8 and new_iteration < state["max_iterations"]:
        next_agent = "writer"
        message = f"[质检] 分数{score}/10, 需要改进"
    else:
        next_agent = "END"
        message = f"[质检] 分数{score}/10, 质量达标"
    
    return {
        **state,
        "messages": [message],
        "next_agent": next_agent,
        "iteration": new_iteration
    }

print("✅ 质量检查 Agent 定义完成！")


✅ 质量检查 Agent 定义完成！


## 四、可视化工作流


In [9]:
# 打印工作流结构
def visualize_workflow():
    print("\n📊 多 Agent 工作流结构:")
    print("\n")
    print("    ┌─────────────┐")
    print("    │   开始      │")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │  研究员      │ ← 收集信息")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │   作家       │ ← 撰写内容")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │   编辑       │ ← 审核改进")
    print("    └──────┬──────┘")
    print("           │")
    print("           ↓")
    print("    ┌─────────────┐")
    print("    │质量检查(可选)│ ← 评估质量")
    print("    └──────┬──────┘")
    print("           │")
    print("      [分数>=8?]")
    print("       ↙     ↘")
    print("     是       否")
    print("     ↓         ↓")
    print("   结束    返回作家")
    print("\n")

visualize_workflow()



📊 多 Agent 工作流结构:


    ┌─────────────┐
    │   开始      │
    └──────┬──────┘
           │
           ↓
    ┌─────────────┐
    │  研究员      │ ← 收集信息
    └──────┬──────┘
           │
           ↓
    ┌─────────────┐
    │   作家       │ ← 撰写内容
    └──────┬──────┘
           │
           ↓
    ┌─────────────┐
    │   编辑       │ ← 审核改进
    └──────┬──────┘
           │
           ↓
    ┌─────────────┐
    │质量检查(可选)│ ← 评估质量
    └──────┬──────┘
           │
      [分数>=8?]
       ↙     ↘
     是       否
     ↓         ↓
   结束    返回作家




## 五、更多实用案例

### 5.1 客服团队 Agent


In [10]:
# 客服团队示例：分类 → 专家 → 质检

class CustomerServiceState(TypedDict):
    query: str
    category: str
    response: str
    satisfaction: str

def classifier_agent(state):
    """分类 Agent：判断问题类型"""
    print("📋 分类 Agent: 分析问题类型...")
    categories = ["技术问题", "账单问题", "产品咨询", "投诉建议"]
    # 简化：直接分类
    return {**state, "category": "技术问题"}

def technical_agent(state):
    """技术支持 Agent"""
    print("🔧 技术支持 Agent: 处理技术问题...")
    return {**state, "response": "这是技术支持的回答..."}

def billing_agent(state):
    """账单 Agent"""
    print("💰 账单 Agent: 处理账单问题...")
    return {**state, "response": "这是账单部门的回答..."}

print("✅ 客服团队 Agent 定义完成！")
print("\n💡 提示: 这个例子展示了如何根据分类路由到不同的专家 Agent")


✅ 客服团队 Agent 定义完成！

💡 提示: 这个例子展示了如何根据分类路由到不同的专家 Agent


### 5.2 代码审查团队


In [11]:
# 代码审查团队：安全审查 + 性能审查 + 风格审查

class CodeReviewState(TypedDict):
    code: str
    security_issues: list
    performance_issues: list
    style_issues: list
    approved: bool

def security_reviewer(state):
    """安全审查 Agent"""
    print("🔒 安全审查 Agent: 检查安全漏洞...")
    # 这里可以集成实际的安全检查工具
    return {**state, "security_issues": []}

def performance_reviewer(state):
    """性能审查 Agent"""
    print("⚡ 性能审查 Agent: 分析性能问题...")
    return {**state, "performance_issues": []}

def style_reviewer(state):
    """代码风格 Agent"""
    print("🎨 风格审查 Agent: 检查代码规范...")
    return {**state, "style_issues": []}

print("✅ 代码审查团队定义完成！")
print("\n💡 提示: 可以并行运行多个审查 Agent，提高效率")


✅ 代码审查团队定义完成！

💡 提示: 可以并行运行多个审查 Agent，提高效率


## 六、总结

### ✅ 掌握的技能

1. **LangGraph 基础**
   - State 状态管理
   - Node 节点定义
   - Edge 边和路由
   - Graph 工作流编排

2. **多 Agent 协作**
   - Agent 角色分工
   - 状态传递和共享
   - 条件路由
   - 循环和迭代

3. **实战模式**
   - 串行协作（研究→写作→编辑）
   - 并行处理（多个审查 Agent）
   - 条件分支（根据分类路由）
   - 质量循环（不达标重做）

### 🚀 应用场景

- 📝 内容创作团队
- 🎯 客户服务系统
- 🔍 代码审查流程
- 📊 数据分析管道
- 🤖 智能助手系统

### 💡 最佳实践

1. **清晰的状态设计**：定义完整的状态结构
2. **单一职责**：每个 Agent 专注一个任务
3. **错误处理**：考虑异常情况和回退机制
4. **性能优化**：并行处理独立任务
5. **可观测性**：添加日志和监控

---

**🎉 恭喜！你已经掌握了 LangGraph 多 Agent 系统的核心技能！**
